In [ ]:
from helpers.db.db_setup_methods import *
from helpers.db.db_query_methods import *

from helpers.db.db_helper_methods import *

init_db(2000000)



So a covering index should be a index that contains all the values needed for the query. And idea is that it should be faster since you dont have to traverse the clustered index to get all the data you need

So for starters i want to test if i select two fields but use only a only a index with a single key that i search for in the db. This should be slower than if it were a covering index even though the where is only on one of the keys

In [ ]:
drop_non_clustered_indexes("products", get_database_name(), output=False)

query = """
CREATE INDEX idx_size ON products(size);
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=False)

query = """
SELECT name, size FROM products WHERE size = 'Large';
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN "+ query, database=get_database_name(), print_as_df=True, show_metrics=False)

In [ ]:
drop_non_clustered_indexes("products", get_database_name(), output=False)

query = """
CREATE INDEX idx_size_and_name ON products(size, name);
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=False)

query = """
SELECT name, size FROM products WHERE size = 'Large';
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN "+ query, database=get_database_name(), print_as_df=True, show_metrics=False)

It seems to be faster. I did have to up the records size to 2m in order to see the difference.

I want to try with also using some sql methods in a covering index

In [ ]:
drop_non_clustered_indexes("products", get_database_name())

query = """
SELECT AVG(price) FROM products WHERE category_id = 1;
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN "+ query, database=get_database_name(), print_as_df=True, show_metrics=False)

In [ ]:
drop_non_clustered_indexes("products", get_database_name())

query = """
CREATE INDEX idx_category_id ON products(category_id);
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=False)

query = """
SELECT AVG(price) FROM products WHERE category_id = 1;
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN "+ query, database=get_database_name(), print_as_df=True, show_metrics=False)

In [ ]:
drop_non_clustered_indexes("products", get_database_name())

query = """
CREATE INDEX idx_category_id_and_price ON products(category_id, price);
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=False)

query = """
SELECT AVG(price) FROM products WHERE category_id = 1;
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN "+ query, database=get_database_name(), print_as_df=True, show_metrics=False)

Here we can see in the extra that it is using the index which just means that it gets all the information it needs from the index and dont have to traverse the clustered index to look at the data. Pretty cool